# **Анализ данных**

##### Задача
В группе компаний Тинькофф есть команда edTech, создающая платформу для обучающих курсов.
У команды edTech возникает вопрос, а какие курсы сильнее всего влияют на рабочие показатели сотрудников в колл-центре. 
Помогите составить рекомендации, какие обучающие курсы стоит проходить сотрудникам, а какие курсы стоит убрать с edTech платформы.
Решение может быть как рекомендательной моделью для каждого из сотрудников, так и основано на бизнес-правилах и статистическом анализе (например, выделить для каждого департамента полезные курсы).

##### Потенциальное решение
Предполагается, что решение будет выполнено на Python, будет итоговая презентация. Решение может содержать блоки: эксплораторный анализ, Causal Inference методы, рекомендательную модель.
Ограничений на подход к решению нет, но для определения важности курсов советуем использовать методы анализа причинности. Подробнее можно почитать о них [здесь,](https://koch-kir.medium.com/causal-inference-from-observational-data-%D0%B8%D0%BB%D0%B8-%D0%BA%D0%B0%D0%BA-%D0%BF%D1%80%D0%BE%D0%B2%D0%B5%D1%81%D1%82%D0%B8-%D0%B0-%D0%B2-%D1%82%D0%B5%D1%81%D1%82-%D0%B1%D0%B5%D0%B7-%D0%B0-%D0%B2-%D1%82%D0%B5%D1%81%D1%82%D0%B0-afb84f2579f2) а также возможна консультация со стороны организаторов.

##### Описание таблиц
###### employees
Информация о сотрудниках колл-центра
Поля:
- employee_id - идентификатор сотрудника
- sex – пол
- region - идентификатор федерального округа
- age – возраст
- head_employee_id – идентификатор руководителя
- exp_days – опыт в днях
- edu_degree – уровень образования
- department_id – индентификатор департамента, в котором работает сотрудник
- work_online_flg – флаг работы на удалённом режиме

###### communications
Информация о рабочих показателях сотрудников. Рассматривались рабочие коммункации операторов колл-центра
Поля:
- communication_id – идентификатор коммуникации
- communication_dt – дата коммуникации
- employee_id - идентификатор сотрудника
- communication_score – оценка качества коммуникации
- util_flg – флаг того, что клиент воспользовался банковским продуктом в течение 2 недель

###### courses_passing
Статиситка прохождения обучающих курсов сотрудниками
- course_id – идентификатор курса
- employee_id - идентификатор сотрудника
- pass_frac – доля прохождения курса
- start_dt – дата начала прохождения
- last_activity_dt – последняя активность сотрудника в обучающем курсе
- end_dt – дата окончания обучения. Если обучение пройдено не полностью, то NaN
- educ_duration_days – длительность полного обучения в днях. Если обучение пройдено не полностью, то NaN

###### courses_info 
Информация о курсах
- course_id – идентификатор курса
- course_nm – название курса

###### course_employee_sms 
Сводная таблица с нотификациями сотрудникам с предложением пройти обучение. Нотификации рассылались случайным образом
Поля:
- employee_id - идентификатор сотрудника
- course_i – флаг наличия нотификации

### **Откроем файл с данными и изучим общую информацию**

In [ ]:
# импортируем библиотеки для обработки данных
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# импортируем другие библиотеки
import warnings

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
# Установка максимального количества отображаемых строк равным None
pd.set_option('display.max_rows', 10)

Попробуем открыть все мастер каталоги и объединить их в отдельный датафрейм

In [ ]:
# Загрузка каждого датасета в отдельный датафрейм
communications_df = pd.read_csv('data\communications.csv', sep=';')
courses_info_df = pd.read_csv('data\courses_info.csv', sep=';')
courses_passing_df = pd.read_csv('data\courses_passing.csv', sep=';')
course_employee_sms_df = pd.read_csv('data\course_employee_sms.csv', sep=';')
employees_df = pd.read_csv('data\employees.csv', sep=';')

In [ ]:
# Расмотрим загруженные данные
communications_df.head()

In [ ]:
# Расмотрим информацию о данных
communications_df.info()

Проверим на пропуски:

In [ ]:
# Проверка наличия пропущенных значений в каждом столбце
communications_df.isnull().sum()

In [ ]:
# Посмотрим количество работников
len(communications_df['employee_id'].unique())

### Анализ эффективности коммуникаций

In [ ]:
# Распределение оценок коммуникации
fig_hist = px.histogram(communications_df, x='communication_score', nbins=20, marginal='box',
                        title='Распределение оценок качества коммуникаций')
fig_hist.update_layout(xaxis_title='Оценка коммуникации', yaxis_title='Количество',
                       height=500)
fig_hist.show()

In [ ]:
# Диаграммма средних оценок коммуникации в зависимости от использования продукта
mean_scores = communications_df.groupby('util_flg', as_index=False)['communication_score'].mean()
fig_bar = px.bar(mean_scores, x='util_flg', y='communication_score', text='communication_score',
                 title='Средняя оценка коммуникации по использованию продукта')
fig_bar.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig_bar.update_layout(xaxis_title='Использование продукта (0 - нет, 1 - да)',
                      yaxis_title='Средняя оценка коммуникации',
                      xaxis_tickmode='array',
                      xaxis_tickvals=[0, 1],
                      xaxis_ticktext=['Не использовали', 'Использовали'],
                      height=600)
fig_bar.show()

In [ ]:
# Считаем корреляцию
correlation_matrix = communications_df[['communication_score', 'util_flg']].corr()

# Создаем тепловую карту
fig_heatmap = go.Figure(data=go.Heatmap(
                   z=correlation_matrix.values,
                   x=correlation_matrix.columns,
                   y=correlation_matrix.columns,
                   hoverongaps=False, colorscale='Viridis'))

# Добавляем аннотации с значениями
annotations = []
for i, row in enumerate(correlation_matrix.values):
    for j, value in enumerate(row):
        annotations.append(go.layout.Annotation(text=str(round(value, 2)), x=correlation_matrix.columns[j], y=correlation_matrix.columns[i],
                                                xref='x1', yref='y1', showarrow=False, font=dict(color="white")))

fig_heatmap.update_layout(annotations=annotations, title='Корреляционная матрица', xaxis_title='', yaxis_title='',
                          height=600)

fig_heatmap.show()

### Временной анализ

In [ ]:
# Преобразование типа данных даты и сортировка
communications_df['communication_dt'] = pd.to_datetime(communications_df['communication_dt'])
communications_df.sort_values('communication_dt', inplace=True)

# Расчет средней оценки коммуникации и доли успешных взаимодействий по датам
daily_avg_score = communications_df.groupby('communication_dt')['communication_score'].mean()
daily_util_rate = communications_df.groupby('communication_dt')['util_flg'].mean()

# Создание графиков
fig = make_subplots(specs=[[{"secondary_y": True}]])

# График средней оценки коммуникации
fig.add_trace(go.Scatter(x=daily_avg_score.index, y=daily_avg_score, name='Средняя оценка коммуникации',
                         line=dict(color='blue', width=2)), secondary_y=False)

# График доли успешных взаимодействий
fig.add_trace(go.Scatter(x=daily_util_rate.index, y=daily_util_rate, name='Доля успешных взаимодействий',
                         line=dict(color='red', width=2, dash='dot')), secondary_y=True)

# Настройка графика
fig.update_layout(title_text='Временной анализ качества коммуникаций и успешности взаимодействий',
                  xaxis_title='Дата',
                  yaxis_title='Средняя оценка коммуникации',
                  template='plotly_white',
                  height=600)

fig.update_yaxes(title_text='Средняя оценка коммуникации', secondary_y=False)
fig.update_yaxes(title_text='Доля успешных взаимодействий', secondary_y=True)

fig.show()

### Сегментация сотрудников

In [ ]:
# Рассчитываем среднюю оценку коммуникации и долю успешных взаимодействий для каждого сотрудника
employee_metrics = communications_df.groupby('employee_id').agg(
    avg_communication_score=('communication_score', 'mean'),
    success_rate=('util_flg', 'mean')
)

In [ ]:
# Нормализация данных
scaler = StandardScaler()
scaled_features = scaler.fit_transform(employee_metrics)

In [ ]:
# Применение K-Means для кластеризации
kmeans = KMeans(n_clusters=3, random_state=12345)
employee_metrics['Cluster'] = kmeans.fit_predict(scaled_features)

In [ ]:
# Визуализация результатов кластеризации
fig = go.Figure()

for cluster in employee_metrics['Cluster'].unique():
    cluster_data = employee_metrics[employee_metrics['Cluster'] == cluster]
    fig.add_trace(go.Scatter(x=cluster_data['avg_communication_score'], y=cluster_data['success_rate'],
                             mode='markers', name=f'Кластер {cluster}',
                             marker=dict(size=10, line=dict(width=2))))

fig.update_layout(title='Сегментация сотрудников по качеству коммуникаций и успешности взаимодействий',
                  xaxis_title='Средняя оценка коммуникации',
                  yaxis_title='Доля успешных взаимодействий',
                  template='plotly_white')

fig.show()